In [2]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigsh

# Создание графа
# edges = [
#     ('jqt', 'rhn'), ('jqt', 'xhk'), ('jqt', 'nvd'),
#     ('rhn', 'xhk'), ('rhn', 'bvb'), ('rhn', 'hfx'),
#     ('bvb', 'xhk'), ('bvb', 'hfx'),
#     ('ntq', 'jqt'), ('ntq', 'hfx'), ('ntq', 'bvb'), ('ntq', 'xhk'),
#     ('xhk', 'hfx'),
#     ('rzs', 'qnr'), ('rzs', 'cmg'), ('rzs', 'lsr'), ('rzs', 'rsh'),
#     ('rsh', 'frs'), ('rsh', 'pzl'), ('rsh', 'lsr'),
#     ('frs', 'qnr'), ('frs', 'lhk'), ('frs', 'lsr'),
#     ('cmg', 'qnr'), ('cmg', 'nvd'), ('cmg', 'lhk'), ('cmg', 'bvb'),
#     ('pzl', 'lsr'), ('pzl', 'hfx'), ('pzl', 'nvd'),
#     ('qnr', 'nvd'),
#     ('lsr', 'lhk'),
#     ('nvd', 'lhk'),
# ]

with open('input.txt') as f:
    file = f.readlines()
    

edges = []
for y, line in enumerate(file):
    [component, elements] = [it.strip() for it in line.rstrip().split(':')]
    for element in [elements.strip() for elements in elements.split(' ')]:
        edges.append((component, element))            

# print(edges)

G = nx.Graph(edges)

# Получение лапласиана графа
L = nx.laplacian_matrix(G).asfptype()

# Вычисление Fiedler вектора
eigenvalues, eigenvectors = eigsh(L, k=2, which='SM')
fiedler_vector = eigenvectors[:, 1]

# Находим три ребра, которые разделят граф

partitions = np.sign(fiedler_vector)
cut_edges = []

# Создание отображения между строковыми метками вершин и числовыми индексами
node_to_index = {node: idx for idx, node in enumerate(G.nodes)}

# Находим ребра с наименьшими весами, соединяющие вершины из разных частей графа
for u, v, data in sorted(G.edges(data=True), key=lambda x: x[2].get('weight', 1)):
    u_index, v_index = node_to_index[u], node_to_index[v]
    if partitions[u_index] != partitions[v_index]:
        cut_edges.append((u, v))

# print("Fiedler Вектор:", fiedler_vector)
print("Три ребра для разделения:", cut_edges)

# Создание двух подграфов
subgraph_1 = G.subgraph([node for node in G.nodes if partitions[node_to_index[node]] == 1])
subgraph_2 = G.subgraph([node for node in G.nodes if partitions[node_to_index[node]] == -1])

# Подсчёт количества рёбер в каждой группе
edges_count_group_1 = subgraph_1.number_of_edges()
edges_count_group_2 = subgraph_2.number_of_edges()

print("Количество рёбер в первой группе:", edges_count_group_1)
print("Количество рёбер во второй группе:", edges_count_group_2)
print("Количество нод в первой группе:", subgraph_1.number_of_nodes())
print("Количество нод во второй группе:", subgraph_2.number_of_nodes())
print("Part I:", subgraph_1.number_of_nodes() * subgraph_2.number_of_nodes())

Три ребра для разделения: [('jxx', 'qdp'), ('qqq', 'mlp'), ('zbr', 'vsx')]
Количество рёбер в первой группе: 1596
Количество рёбер во второй группе: 1698
Количество нод в первой группе: 712
Количество нод во второй группе: 763
Part I: 543256


C:\Users\kobza\AppData\Local\Temp\ipykernel_10668\3273264703.py:37: DeprecationWarning: `asfptype` is an internal function, and is deprecated as part of the public API. It will be removed in v1.13.0.
  L = nx.laplacian_matrix(G).asfptype()
